<a href="https://colab.research.google.com/github/kdemertzis/EKPA/blob/main/Lambda_IDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#προσαρμογή από IDS σε IPS

In [87]:
#εγκατάσταση πακέτου tensorflow (https://www.tensorflow.org/)
!pip install tensorflow
#εγκατάσταση πακέτου scikit-learn (https://scikit-learn.org/)
!pip install scikit-learn

In [88]:
#εισαγωγή βιβλιοθηκών - πακέτων
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, LSTM

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [89]:
#αριθμός κλάσεων και epochs
nclass = 12
epochs = 5 #για συντομία 5

#batch training με IPS
batch_size = 10000

In [66]:
#συνάρτηση εισαγωγής συνόλου δεδομένων σε δύο κλάσεις Νormal(0) και Attack(1)
def load_dataset(file_path):
    df = pd.read_csv(file_path)
    X = df.drop('target', axis=1).values
    y = df['target'].apply(lambda x: 0 if x == "normal." else 1)
    return X, y

In [90]:
#επιλογή αρχιτεκτονικής Dense, Conv1D, LSTM
architecture_choice='Dense'

In [91]:
#συνάρτηση δημιουργίας IPS μοντέλου
def create_and_train_model(input_dim, nclass, architecture_choice):
    model = Sequential()

    #επιλογή μεταξύ μοντέλων αρχιτεκτονικής
    if  architecture_choice == 'Dense':
            model.add(Dense(256, input_dim=input_dim, activation='relu'))
            model.add(Dropout(0.5))
            model.add(Dense(128, activation='relu'))
            model.add(Dropout(0.5))
            model.add(Dense(64, activation='relu'))
    elif architecture_choice == 'Conv1D':
            model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(input_dim,1)))
            model.add(Flatten())
            model.add(Dense(128, activation='relu'))
            model.add(Dropout(0.5))
    elif architecture_choice == 'LSTM':
            model.add(LSTM(64, input_shape=(input_dim,1)))
            model.add(Dense(128, activation='relu'))
    model.add(Dense(nclass, activation='softmax')) #multi-class output
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [92]:
#συνάρτηση IPS απόφασης
def ips_decision(y_true, y_pred_probs, blocking_threshold=0.9):
    y_pred = np.argmax(y_pred_probs, axis=1)
    blocked = []

    for i, (true_label, probs) in enumerate(zip(y_true, y_pred_probs)):
      attack_prob = max(probs[1:])
      if attack_prob > blocking_threshold:
        blocked.append(i)
        print(f"[IPS Aktion] Blocking traffic instance {i}, isolating device, notifying admin.")
    return y_pred, blocked

In [93]:
#θέση αρχείου csv με το σύνολο δεδομένων
csv_file_path = 'https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/pcap_data.csv'

In [94]:
#τυποποίηση (standardize) δεδομένων
scaler = StandardScaler()

In [95]:
#φόρτωση συνόλου δεδομένων
X, y = load_dataset(csv_file_path)

#τυποποίηση (standardize) δεδομένων
X = scaler.fit_transform(X)

In [96]:
#split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [97]:
#αρχικοποίηση μοντέλου
model = create_and_train_model(X_train.shape[1], nclass, architecture_choice)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [100]:
#εκπαίδευση μοντέλου με αυξητική μάθηση

for epoch in range(1, epochs+1):
  print(f"\nEpoch {epoch}/{epochs}")

  for i in range(0, len(X_train), batch_size):
    X_batch = X_train[i:i+batch_size]
    y_batch = y_train[i:i+batch_size]

    #εκπαίδευση σε κάθε batch
    model.fit(X_batch, y_batch)

    #περιοδική ενημέρωση του μοντέλου με κάθε ροή (batch) δεδομένων
    y_pred_probs = model.predict(X_test, verbose=0)
    y_pred, blocked_instances = ips_decision(y_test, y_pred_probs, blocking_threshold=0.9)

    acc = accuracy_score(y_test, y_pred)
    print(f"Epoch {epoch}, Iteration {i}, Accuracy: {acc:.4f}, Blocked: {len(blocked_instances)}")


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
[IPS Aktion] Blocking traffic instance 993, isolating device, notifying admin.
[IPS Aktion] Blocking traffic instance 994, isolating device, notifying admin.
[IPS Aktion] Blocking traffic instance 995, isolating device, notifying admin.
[IPS Aktion] Blocking traffic instance 996, isolating device, notifying admin.
[IPS Aktion] Blocking traffic instance 997, isolating device, notifying admin.
[IPS Aktion] Blocking traffic instance 998, isolating device, notifying admin.
[IPS Aktion] Blocking traffic instance 999, isolating device, notifying admin.
[IPS Aktion] Blocking traffic instance 1000, isolating device, notifying admin.
[IPS Aktion] Blocking traffic instance 1001, isolating device, notifying admin.
[IPS Aktion] Blocking traffic instance 1002, isolating device, notifying admin.
[IPS Aktion] Blocking traffic instance 1003, isolating device, notifying admin.
[IPS Aktion] Blocking traffic instance 1004, isolating devic

In [101]:
#τελική αξιολόγηση
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print(f"Total Blocked Instances: {len(blocked_instances)}")


Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00      2994

    accuracy                           1.00      2994
   macro avg       1.00      1.00      1.00      2994
weighted avg       1.00      1.00      1.00      2994

Total Blocked Instances: 2994
